# Waves

<p style="font-family: arial, Times New Roman; font-size:12pt; color:black;text-align:justify">
Info.  Buoy series in .mat file with variables of (year, month, day, hour, hs, tm, tp, dir, dirp, sprp, spr) in columns<br>
    To ease the analysis, it is recommended to use the scipy library to read data and DataFrame format for graphs (pandas, matplotlib, plotly, plotly express) This notebook uses plotly express library, for more information, please refer to the plotly express documentation 
    <a href="https://plotly.com/python/plotly-express/">https://plotly.com/python/plotly-express/</a>
</p>

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# system
import os
import os.path as op
import sys

# arrays
import numpy as np
import pandas as pd
from scipy.io import loadmat
import datetime

# matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# plotly 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 3.1 Load data

In [ ]:
p_db = op.join(os.getcwd(),'..','..','data','Buoys')

# database waves (mat file)
p_dat = op.join(p_db, 'Bilbao-Vizcaya Ext.mat')

# database exploration
data_mat = loadmat(p_dat)

array = data_mat[list(data_mat.keys())[-1]]

In [ ]:
data = pd.DataFrame(
    {
        'year': array[:,0],
        'month': array[:,1],
        'day': array[:,2],
        'hour': array[:,3],
        'hs': array[:,4],
        'tm': array[:,5],
        'tp': array[:,6],
        'dir': array[:,9],
        'dirp': array[:,10],
        'sprp': array[:,11],
        'spr': array[:,12],
    }
)
data

In [ ]:
data['dates'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data = data.drop(['year', 'month', 'day', 'hour'], axis=1).set_index('dates')
data

In [ ]:

data.boxplot(by = data.index.month, column = 'hs', figsize = (15,12))

## 3.2 Plot series

In [ ]:
# Use pandas library for plotting daily series
data.plot(
    subplots=True,
    figsize=(17,14), layout=(7,1),
    color=['royalblue','gold','orangered','skyblue','seagreen','thistle']
)
plt.show()

## 3.3. Scatter matrix of Energy flux

<p style="font-family: arial, Times New Roman; font-size:12pt; color:black;text-align:justify">
    Energy flux measured by F = Hs^2 * Tp. Scatter plot with energy flux color<br>
    <a href="https://plotly.com/python/splom/">Plotly express library</a>

</p>

In [ ]:
# create dataframe column with energy flux EF values
data['EF'] = data['hs']**2 * data['tp']
data

In [ ]:
import plotly.express as px

fig = px.scatter_3d(data.dropna(), x='hs', y='tp', z='dir', size='hs', size_max=10, color='EF')
fig.update_traces(marker=dict(size=3,
                              line=dict(width=0)),
                  selector=dict(mode='markers'))

fig.show()

In [ ]:
fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='Hs (m)',
                                 values=data['hs']),
                            dict(label='Tm (s)',
                                 values=data['tm']),
                            dict(label='Tp (s)',
                                 values=data['tp']),
                            dict(label='Dir (º)',
                                 values=data['dir']),
                            dict(label='DirP (º)',
                                 values=data['dirp']),
                            dict(label='Spr (º)',
                                 values=data['spr']),
                            dict(label='SprP (º)',
                                 values=data['sprp'])],
                showlowerhalf=False, # remove plots on diagonal
                diagonal=dict(visible=False),
                text=data['EF'],
                marker=dict(color=data['EF'],
                            colorscale='magma_r',
                            showscale=True, # colorbar
                            size=2)
                ))


fig.update_layout(
    title='Scatter matrix of Buoy variables',
    width=1100,
    height=1100,
)

fig.show()

## 3.4 Density contour Hs-Tp

In [ ]:
fig = px.density_contour(data, x="hs", y="tp", marginal_x="histogram", marginal_y="box")
fig.update_layout(
    width=700,
    height=700,
)
fig.show()

## 3.5 Density heatmap Hs-Tp

In [ ]:
fig = px.density_heatmap(data, x="hs", y="dir", marginal_x="histogram", marginal_y="histogram")
fig.update_layout(
    width=700,
    height=700,
)
fig.show()

## 3.6 Gumbel fit to Hs and Tp variables

In [ ]:
import scipy.stats as ss

nbins = 30
t = np.linspace(-1,data.hs.max(),100)
t1 = np.linspace(0,data.tp.max()+5,100)

fig, axs = plt.subplots(1,3, figsize=(13,5))

# Hs fit
fit = ss.gumbel_r.fit(data.hs)
pdf_gumbel = ss.gumbel_r.pdf(t, *fit)

data.hs.hist(
    ax=axs[0],rwidth=0.9,
    bins=nbins, density=1, 
    color='b', alpha=0.5,
    label='Hs (m)'
)

# Tp fit
fit1 = ss.gumbel_r.fit(data.tp)
pdf_gumbel1 = ss.gumbel_r.pdf(t1, *fit1)

data.tp.hist(
    ax=axs[1], rwidth=0.9,
    bins=nbins, density=1, 
    color='b', alpha=0.5,
    label='Tp (s)'
)
axs[0].set_ylabel('Probability density')

axs[0].legend()
axs[1].legend()

# Tp fit
fit1 = ss.weibull_min.fit(data.tp)
pdf_weibull1 = ss.weibull_min.pdf(t1, *fit1)

data.tp.hist(
    ax=axs[2], rwidth=0.9,
    bins=nbins, density=1, 
    color='b', alpha=0.5,
    label='Tp (s)'
)
axs[0].plot(t, pdf_gumbel, 'k', label='Gumbel Fit')
axs[1].plot(t1, pdf_gumbel1, 'k', label='Gumbel Fit')
axs[2].plot(t1, pdf_weibull1, 'k', label='Weibull Fit')

axs[0].set_ylabel('Probability density')

axs[0].legend()
axs[1].legend()
axs[2].legend()

plt.show()